# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
total_customer_purchase = data.groupby(by=['CustomerID','ProductID','ProductName']).sum().reset_index(drop=False)
total_customer_purchase.head()

,CustomerID,ProductID,ProductName,SalesID,Quantity
0,33,23,Crab - Imitation Flakes,6515659,1
1,33,51,Wine - Crozes Hermitage E.,5332057,1
2,33,53,Cassis,1888258,1
3,33,67,Wine - Sogrape Mateus Rose,3975703,1
4,33,70,Squid - Tubes / Tenticles 10/20,2235546,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
recommend = pd.pivot_table(total_customer_purchase, values='Quantity', index='ProductName', columns='CustomerID').fillna(0)
recommend.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
distances = pdist(recommend.T,'euclidean')
distances_matrix = pd.DataFrame(squareform(distances),columns=recommend.columns, index=recommend.columns)
distances = pd.DataFrame(1/(1 + distances_matrix), index=recommend.columns, columns=recommend.columns)
display(distances)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
def similar(customer):
    return distances[customer].sort_values()[1:]
similar(412)[:5]

CustomerID
97324    0.004035
95017    0.004087
92492    0.004131
91777    0.004160
92525    0.004176
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
def purchaseSimilar(customer):
    return total_customer_purchase[total_customer_purchase['CustomerID'].isin(list(similar(customer)[:5].index))]
purchase_similar_33 = purchaseSimilar(33)
purchase_similar_33.head()

,CustomerID,ProductID,ProductName,SalesID,Quantity
59478,91777,3,Onions - Cippolini,4257147,24
59479,91777,10,Scampi Tail,2125244,24
59480,91777,11,Garbage Bags - Clear,2793573,24
59481,91777,12,Ezy Change Mophandle,4000213,24
59482,91777,17,"Wine - White, Mosel Gold",6252822,24


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [12]:
def recommend_cust(customer): 
    return purchaseSimilar(customer).groupby(by='ProductName').sum().sort_values(by='Quantity',ascending=False)
recommend_33 = recommend_cust(33)
recommend_33

,CustomerID,ProductID,SalesID,Quantity
ProductName,,,,
"Turnip - White, Organic",371811,208,17623258,146
"Beef - Tenderlion, Center Cut",279286,1056,12597850,122
Whmis - Spray Bottle Trigger,279319,1260,14013453,98
Pastry - Butterscotch Baked,187509,118,11832037,98
Clam Nectar,187509,532,8891668,98
...,...,...,...,...
Ice Cream Bar - Oreo Cone,91777,413,3089475,24
Ice Cream Bar - Drumstick,92525,154,5567458,24
Hot Chocolate - Individual,91777,351,2716494,24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [13]:
def recommendation(customer):
    return list(recommend_cust(customer).filter(items=(recommend[customer] == 0).index).index)
recommend_33_top_5 = recommendation(33)[:5]
recommend_33_top_5

['Turnip - White, Organic',
 'Beef - Tenderlion, Center Cut',
 'Whmis - Spray Bottle Trigger',
 'Pastry - Butterscotch Baked',
 'Clam Nectar']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [17]:
recommendations = {}
customers = list(distances_matrix.columns)
for cust in customers:
    recommendations[cust] = recommendation(cust)[:5]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.iloc[:,0:5]

,33,200,264,356,412
0,"Turnip - White, Organic","Turnip - White, Organic","Turnip - White, Organic","Turnip - White, Organic","Turnip - White, Organic"
1,"Beef - Tenderlion, Center Cut",Whmis - Spray Bottle Trigger,"Beef - Tenderlion, Center Cut","Beef - Tenderlion, Center Cut","Beef - Tenderlion, Center Cut"
2,Whmis - Spray Bottle Trigger,"Beef - Tenderlion, Center Cut",Whmis - Spray Bottle Trigger,Whmis - Spray Bottle Trigger,Whmis - Spray Bottle Trigger
3,Pastry - Butterscotch Baked,Bread - Hot Dog Buns,Pastry - Butterscotch Baked,Pastry - Butterscotch Baked,Pastry - Butterscotch Baked
4,Clam Nectar,Pastry - Raisin Muffin - Mini,Clam Nectar,Clam Nectar,Clam Nectar


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [19]:
distances = pdist(recommend.T,'cosine')
distances_matrix = pd.DataFrame(squareform(distances),columns=recommend.columns, index=recommend.columns)
distances = pd.DataFrame(1/(1 + distances_matrix), index=recommend.columns, columns=recommend.columns)
recommendations_correlation = {}
customers = list(distances_matrix.columns)
for cust in customers:
    recommendations_correlation[cust] = recommendation(cust)[:5]
recommendations_correlation_df = pd.DataFrame(recommendations_correlation)
recommendations_correlation_df.iloc[:,0:5]

,33,200,264,356,412
0,Cinnamon Buns Sticky,"Pepper - White, Ground",Beer - Original Organic Lager,Snapple Lemon Tea,Foam Cup 6 Oz
1,Juice - V8 Splash,Wine - White Cab Sauv.on,"Pork - Bacon, Double Smoked",Pail For Lid 1537,Baking Powder
2,Longos - Chicken Wings,Peas - Frozen,Lime Cordial - Roses,Chocolate - Feathers,Peas - Frozen
3,Table Cloth 54x72 White,Arizona - Green Tea,Wine - Magnotta - Belpaese,Lettuce - Spring Mix,Rambutan
4,Tea - English Breakfast,Beef - Rib Eye Aaa,Lettuce - Treviso,Ice Cream Bar - Hageen Daz To,Extract - Lemon
